In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import os
import glob

In [2]:
folder_path = '../data/train_timelines/'
file_list = glob.glob(os.path.join(folder_path, '*.json'))

In [3]:
participantid = []
currentgold = []
totalgold = []
minionskilled = []
xp = []
matchid = []
teamid = []



def file_buster(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
    matchida= data['matchId']
    for i, j in data['frames'][-1]['participantFrames'].items():  #looks at last frame, looks at participantFrames equiv to last_stats
        participantid.append(i)
        currentgold.append(j['currentGold'])
        totalgold.append(j['totalGold'])
        xp.append(j['xp'])
        minionskilled.append(j['minionsKilled'])
        matchid.append(matchida)
        if int(i) < 6:
            teamid.append('100')
        else:
            teamid.append('200')


In [4]:
count = 0
for file in file_list:
    #print('loading file: ' + str(count))
    file_buster(file)
    count+=1
print(str(count) + " files loaded")

8000 files loaded


In [5]:
di = {'matchId':matchid, 'teamId':teamid, 'participantId' : participantid, 
      'currentGold': currentgold, 'totalGold':totalgold, 'xp': xp, 'minionsKilled': minionskilled}
z_train = pd.DataFrame(di)
z_train

,matchId,teamId,participantId,currentGold,totalGold,xp,minionsKilled
0,0,100,1,341,2741,4141,62
1,0,100,2,5,3045,2861,4
2,0,100,3,365,3545,4712,78
3,0,100,4,1214,2789,2894,62
4,0,100,5,566,2026,2551,4
...,...,...,...,...,...,...,...
79995,999,200,6,1127,1127,851,7
79996,999,200,7,844,1694,1546,1
79997,999,200,8,56,1731,2489,37
79998,999,200,9,212,1512,1257,27


Total Gold Difference

In [9]:
gold_data = pd.DataFrame(z_train.groupby(['matchId', 'teamId'])['totalGold'].sum()).reset_index()

gold_data_piv = gold_data.pivot(index = 'matchId', columns = 'teamId', values = 'totalGold').reset_index()
gold_data_piv['gold_difference'] = gold_data_piv['100'] - gold_data_piv['200']
gold_data_piv.columns = ['matchId', 't1totG', 't2totG', 'totGdif']
gold_data_piv

,matchId,t1totG,t2totG,totGdif
0,0,14146,17539,-3393
1,1,17662,14816,2846
2,2,16374,17347,-973
3,3,19247,15145,4102
4,4,16776,15385,1391
...,...,...,...,...
7995,7995,19009,14873,4136
7996,7996,16413,16140,273
7997,7997,17745,16571,1174
7998,7998,15543,18311,-2768


Current Gold Data

In [12]:
curgold_data = pd.DataFrame(z_train.groupby(['matchId', 'teamId'])['currentGold'].sum()).reset_index()
curgold_data
curgold_data_piv = curgold_data.pivot(index = 'matchId', columns = 'teamId', values = 'currentGold').reset_index()
curgold_data_piv['curgold_difference'] = curgold_data_piv['100'] - curgold_data_piv['200']
curgold_data_piv.columns = ['matchId', 't1curG', 't2curG', 'curGdif']
curgold_data_piv

,matchId,t1curG,t2curG,curGdif
0,0,2491,3464,-973
1,1,4382,2861,1521
2,2,2109,2461,-352
3,3,5292,4315,977
4,4,4281,2290,1991
...,...,...,...,...
7995,7995,3054,1803,1251
7996,7996,3208,2980,228
7997,7997,3035,2066,969
7998,7998,2973,4435,-1462


xp data

In [14]:
xp_data = pd.DataFrame(z_train.groupby(['matchId', 'teamId'])['xp'].mean()).reset_index()

xp_data_piv = xp_data.pivot(index = 'matchId', columns = 'teamId', values = 'xp').reset_index()
xp_data_piv['xp_diff'] = xp_data_piv['100'] - xp_data_piv['200']
xp_data_piv.columns = ['matchId', 't1xp', 't2xp', 'xpdif']
xp_data_piv

,matchId,t1xp,t2xp,xpdif
0,0,3431.8,3754.6,-322.8
1,1,3554.0,3605.6,-51.6
2,2,3367.4,3493.4,-126.0
3,3,3512.4,3239.2,273.2
4,4,3652.8,3438.8,214.0
...,...,...,...,...
7995,7995,3760.6,3395.4,365.2
7996,7996,3459.6,3585.4,-125.8
7997,7997,3418.0,3144.2,273.8
7998,7998,3170.6,3730.8,-560.2


minionsKilled

In [16]:
mk_data = pd.DataFrame(z_train.groupby(['matchId', 'teamId'])['minionsKilled'].sum()).reset_index()
#xp_data
mk_data_piv = mk_data.pivot(index = 'matchId', columns = 'teamId', values = 'minionsKilled').reset_index()
mk_data_piv['mk_dif'] = mk_data_piv['100'] - mk_data_piv['200']
mk_data_piv.columns = ['matchId', 't1mk', 't2mk', 'mkdif']
mk_data_piv

,matchId,t1mk,t2mk,mkdif
0,0,210,232,-22
1,1,219,236,-17
2,2,183,195,-12
3,3,194,184,10
4,4,228,218,10
...,...,...,...,...
7995,7995,248,198,50
7996,7996,209,220,-11
7997,7997,196,194,2
7998,7998,185,257,-72


In [31]:
champion_mastery = pd.read_csv('../data/champion_mastery.csv')
participants_train = pd.read_csv('../data/participants_train.csv')

In [33]:
champion_points = pd.merge(participants_train, champion_mastery, on = ['summonerId', 'championId'], how = 'left' ).drop(['chestGranted', 'tokensEarned'], axis = 1)
champion_points

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints
0,0,100,1,0,303,Mordekaiser,82,6.0,61890.0
1,0,100,2,1,616,Sylas,517,5.0,37664.0
2,0,100,3,2,667,Lissandra,127,6.0,40303.0
3,0,100,4,3,860,Caitlyn,51,6.0,96304.0
4,0,100,5,4,325,Morgana,25,7.0,608721.0
...,...,...,...,...,...,...,...,...,...
79995,7999,200,6,13979,595,Yorick,83,6.0,48037.0
79996,7999,200,7,39643,38,Volibear,106,1.0,135.0
79997,7999,200,8,5570,498,Anivia,34,7.0,49027.0
79998,7999,200,9,10228,733,Twitch,29,7.0,174089.0


filtering for maximum champion points within each team

In [34]:
max_champ_pts = pd.DataFrame(champion_points.groupby(['matchId', 'teamId'])['championPoints'].max().reset_index())
max_champ_pts
max_champ_pts = pd.merge(champion_points, max_champ_pts, on = ['matchId', 'teamId', 'championPoints']).drop(['summonerLevel'], axis =1)
max_champ_pts

,matchId,teamId,participantId,summonerId,championName,championId,championLevel,championPoints
0,0,100,5,4,Morgana,25,7.0,608721.0
1,0,200,6,5,Warwick,19,7.0,1180052.0
2,1,100,2,11,MasterYi,11,7.0,890336.0
3,1,200,7,16,Gragas,79,7.0,423591.0
4,2,100,4,23,Caitlyn,51,7.0,275745.0
...,...,...,...,...,...,...,...,...
15995,7997,200,10,39642,Thresh,412,7.0,401168.0
15996,7998,100,2,2810,Evelynn,28,7.0,119689.0
15997,7998,200,8,7150,Fizz,105,7.0,385174.0
15998,7999,100,5,2223,Pyke,555,7.0,3297902.0


In [39]:
#dataframe for max champ points by team
max_champ_pts_piv = pd.pivot(data = max_champ_pts, index = 'matchId', columns = 'teamId', values = 'championPoints').reset_index()
#max_champ_pts_piv = max_champ_pts_piv.rename(columns = {'100':'t_100', '200':'t_200'}, inplace = True)
max_champ_pts_piv.columns = ['matchId', 't1maxChPt', 't2maxChPt']
max_champ_pts_piv['mxchmpPtDiff'] = max_champ_pts_piv['t1maxChPt'] - max_champ_pts_piv['t2maxChPt']
max_champ_pts_piv

,matchId,t1maxChPt,t2maxChPt,mxchmpPtDiff
0,0,608721.0,1180052.0,-571331.0
1,1,890336.0,423591.0,466745.0
2,2,275745.0,493275.0,-217530.0
3,3,499057.0,520460.0,-21403.0
4,4,216801.0,49670.0,167131.0
...,...,...,...,...
7995,7995,641676.0,775272.0,-133596.0
7996,7996,544546.0,355696.0,188850.0
7997,7997,298269.0,401168.0,-102899.0
7998,7998,119689.0,385174.0,-265485.0


In [40]:
#selecting champion with highest points within each team
champion = pd.pivot(data = max_champ_pts, index = 'matchId', columns = 'teamId', values = 'championName').reset_index()
champion.columns = ['matchId', 't1champ', 't2champ']
champion

,matchId,t1champ,t2champ
0,0,Morgana,Warwick
1,1,MasterYi,Gragas
2,2,Caitlyn,Jax
3,3,Swain,Kaisa
4,4,Irelia,Lulu
...,...,...,...
7995,7995,JarvanIV,Vayne
7996,7996,Thresh,LeeSin
7997,7997,Urgot,Thresh
7998,7998,Evelynn,Fizz


### Merging tot_gold, curr_gold, xp, minionskilled, maxchamp points, champion with max points

In [41]:
feats = pd.merge(gold_data_piv, curgold_data_piv, on = 'matchId')\
.merge(xp_data_piv, on = 'matchId')\
.merge(mk_data_piv, on = 'matchId')\
.merge(max_champ_pts_piv, on = 'matchId')\
.merge(champion, on = 'matchId')

feats

,matchId,t1totG,t2totG,totGdif,t1curG,t2curG,curGdif,t1xp,t2xp,xpdif,t1mk,t2mk,mkdif,t1maxChPt,t2maxChPt,mxchmpPtDiff,t1champ,t2champ
0,0,14146,17539,-3393,2491,3464,-973,3431.8,3754.6,-322.8,210,232,-22,608721.0,1180052.0,-571331.0,Morgana,Warwick
1,1,17662,14816,2846,4382,2861,1521,3554.0,3605.6,-51.6,219,236,-17,890336.0,423591.0,466745.0,MasterYi,Gragas
2,2,16374,17347,-973,2109,2461,-352,3367.4,3493.4,-126.0,183,195,-12,275745.0,493275.0,-217530.0,Caitlyn,Jax
3,3,19247,15145,4102,5292,4315,977,3512.4,3239.2,273.2,194,184,10,499057.0,520460.0,-21403.0,Swain,Kaisa
4,4,16776,15385,1391,4281,2290,1991,3652.8,3438.8,214.0,228,218,10,216801.0,49670.0,167131.0,Irelia,Lulu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,19009,14873,4136,3054,1803,1251,3760.6,3395.4,365.2,248,198,50,641676.0,775272.0,-133596.0,JarvanIV,Vayne
7996,7996,16413,16140,273,3208,2980,228,3459.6,3585.4,-125.8,209,220,-11,544546.0,355696.0,188850.0,Thresh,LeeSin
7997,7997,17745,16571,1174,3035,2066,969,3418.0,3144.2,273.8,196,194,2,298269.0,401168.0,-102899.0,Urgot,Thresh
7998,7998,15543,18311,-2768,2973,4435,-1462,3170.6,3730.8,-560.2,185,257,-72,119689.0,385174.0,-265485.0,Evelynn,Fizz


In [42]:
train_winners = pd.read_csv('../data/train_winners.csv')
train_winners.head()

,matchId,winner
0,0,200
1,1,200
2,2,200
3,3,100
4,4,200


In [43]:
feats_wins = pd.merge(feats, train_winners, on='matchId')
feats_wins

,matchId,t1totG,t2totG,totGdif,t1curG,t2curG,curGdif,t1xp,t2xp,xpdif,t1mk,t2mk,mkdif,t1maxChPt,t2maxChPt,mxchmpPtDiff,t1champ,t2champ,winner
0,0,14146,17539,-3393,2491,3464,-973,3431.8,3754.6,-322.8,210,232,-22,608721.0,1180052.0,-571331.0,Morgana,Warwick,200
1,1,17662,14816,2846,4382,2861,1521,3554.0,3605.6,-51.6,219,236,-17,890336.0,423591.0,466745.0,MasterYi,Gragas,200
2,2,16374,17347,-973,2109,2461,-352,3367.4,3493.4,-126.0,183,195,-12,275745.0,493275.0,-217530.0,Caitlyn,Jax,200
3,3,19247,15145,4102,5292,4315,977,3512.4,3239.2,273.2,194,184,10,499057.0,520460.0,-21403.0,Swain,Kaisa,100
4,4,16776,15385,1391,4281,2290,1991,3652.8,3438.8,214.0,228,218,10,216801.0,49670.0,167131.0,Irelia,Lulu,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,19009,14873,4136,3054,1803,1251,3760.6,3395.4,365.2,248,198,50,641676.0,775272.0,-133596.0,JarvanIV,Vayne,200
7996,7996,16413,16140,273,3208,2980,228,3459.6,3585.4,-125.8,209,220,-11,544546.0,355696.0,188850.0,Thresh,LeeSin,100
7997,7997,17745,16571,1174,3035,2066,969,3418.0,3144.2,273.8,196,194,2,298269.0,401168.0,-102899.0,Urgot,Thresh,200
7998,7998,15543,18311,-2768,2973,4435,-1462,3170.6,3730.8,-560.2,185,257,-72,119689.0,385174.0,-265485.0,Evelynn,Fizz,200


# LOG REGRESSION MODEL

In [47]:
X = feats_wins.drop(['winner'], 1)
y = feats_wins['winner']

In [54]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [49]:
le = LabelEncoder()

for col in X.columns.values:
    le.fit(X[col].values)
    X[col] = le.transform(X[col])
    
print(X.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 0 to 7999
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   matchId       8000 non-null   int64
 1   t1totG        8000 non-null   int64
 2   t2totG        8000 non-null   int64
 3   totGdif       8000 non-null   int64
 4   t1curG        8000 non-null   int64
 5   t2curG        8000 non-null   int64
 6   curGdif       8000 non-null   int64
 7   t1xp          8000 non-null   int64
 8   t2xp          8000 non-null   int64
 9   xpdif         8000 non-null   int64
 10  t1mk          8000 non-null   int64
 11  t2mk          8000 non-null   int64
 12  mkdif         8000 non-null   int64
 13  t1maxChPt     8000 non-null   int64
 14  t2maxChPt     8000 non-null   int64
 15  mxchmpPtDiff  8000 non-null   int64
 16  t1champ       8000 non-null   int32
 17  t2champ       8000 non-null   int32
dtypes: int32(2), int64(16)
memory usage: 1.1 MB
None


In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.25,
                                                    random_state = 12,
                                                    stratify = y)

In [91]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=39)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [92]:
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[716, 302],
       [296, 686]], dtype=int64)

In [93]:
target_names = ['100', '200']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         100       0.71      0.70      0.71      1018
         200       0.69      0.70      0.70       982

    accuracy                           0.70      2000
   macro avg       0.70      0.70      0.70      2000
weighted avg       0.70      0.70      0.70      2000



# APPLYING TO TEST DATA FOR SUBMISSION

In [66]:
folder_path = '../data/test_timelines/'
file_list = glob.glob(os.path.join(folder_path, '*.json'))

In [67]:
participantid = []
currentgold = []
totalgold = []
minionskilled = []
xp = []
matchid = []
teamid = []



def file_buster(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
    matchida= data['matchId']
    for i, j in data['frames'][-1]['participantFrames'].items():  #looks at last frame, looks at participantFrames equiv to last_stats
        participantid.append(i)
        currentgold.append(j['currentGold'])
        totalgold.append(j['totalGold'])
        xp.append(j['xp'])
        minionskilled.append(j['minionsKilled'])
        matchid.append(matchida)
        if int(i) < 6:
            teamid.append('100')
        else:
            teamid.append('200')


In [68]:
count = 0
for file in file_list:
    #print('loading file: ' + str(count))
    file_buster(file)
    count+=1
print(str(count) + " files loaded")

2000 files loaded


In [69]:
di = {'matchId':matchid, 'teamId':teamid, 'participantId' : participantid, 
      'currentGold': currentgold, 'totalGold':totalgold, 'xp': xp, 'minionsKilled': minionskilled}
z_test = pd.DataFrame(di)
z_test

,matchId,teamId,participantId,currentGold,totalGold,xp,minionsKilled
0,8000,100,1,1513,3063,4659,77
1,8000,100,2,1742,4117,3539,3
2,8000,100,3,1015,4365,4431,70
3,8000,100,4,80,3555,3061,58
4,8000,100,5,78,2213,2409,2
...,...,...,...,...,...,...,...
19995,9999,200,6,217,2852,4621,52
19996,9999,200,7,1510,2860,2905,5
19997,9999,200,8,495,3245,4214,54
19998,9999,200,9,283,2483,2565,51


In [70]:
gold_dataT = pd.DataFrame(z_test.groupby(['matchId', 'teamId'])['totalGold'].sum()).reset_index()

gold_data_pivT = gold_dataT.pivot(index = 'matchId', columns = 'teamId', values = 'totalGold').reset_index()
gold_data_pivT['gold_difference'] = gold_data_pivT['100'] - gold_data_pivT['200']
gold_data_pivT.columns = ['matchId', 't1totG', 't2totG', 'totGdif']
gold_data_pivT

,matchId,t1totG,t2totG,totGdif
0,8000,17313,16933,380
1,8001,17492,17533,-41
2,8002,16206,17480,-1274
3,8003,17784,18126,-342
4,8004,16341,16993,-652
...,...,...,...,...
1995,9995,15440,15060,380
1996,9996,19123,17433,1690
1997,9997,14719,17129,-2410
1998,9998,18353,15134,3219


In [71]:
curgold_dataT = pd.DataFrame(z_test.groupby(['matchId', 'teamId'])['currentGold'].sum()).reset_index()
curgold_dataT
curgold_data_pivT = curgold_dataT.pivot(index = 'matchId', columns = 'teamId', values = 'currentGold').reset_index()
curgold_data_pivT['curgold_difference'] = curgold_data_pivT['100'] - curgold_data_pivT['200']
curgold_data_pivT.columns = ['matchId', 't1curG', 't2curG', 'curGdif']
curgold_data_pivT

,matchId,t1curG,t2curG,curGdif
0,8000,4428,3093,1335
1,8001,3052,3993,-941
2,8002,3171,3265,-94
3,8003,5019,3271,1748
4,8004,4111,3013,1098
...,...,...,...,...
1995,9995,3975,3140,835
1996,9996,4939,2288,2651
1997,9997,4379,4154,225
1998,9998,4063,3159,904


In [72]:
xp_dataT = pd.DataFrame(z_test.groupby(['matchId', 'teamId'])['xp'].mean()).reset_index()

xp_data_pivT = xp_dataT.pivot(index = 'matchId', columns = 'teamId', values = 'xp').reset_index()
xp_data_pivT['xp_diff'] = xp_data_pivT['100'] - xp_data_pivT['200']
xp_data_pivT.columns = ['matchId', 't1xp', 't2xp', 'xpdif']
xp_data_pivT

,matchId,t1xp,t2xp,xpdif
0,8000,3619.8,3423.4,196.4
1,8001,3351.2,3305.4,45.8
2,8002,3328.0,3303.6,24.4
3,8003,3351.8,3434.6,-82.8
4,8004,3675.4,3422.6,252.8
...,...,...,...,...
1995,9995,3537.6,3569.0,-31.4
1996,9996,3654.6,3709.0,-54.4
1997,9997,3442.2,3777.6,-335.4
1998,9998,3659.6,3364.6,295.0


In [73]:
mk_dataT = pd.DataFrame(z_test.groupby(['matchId', 'teamId'])['minionsKilled'].sum()).reset_index()
#xp_data
mk_data_pivT = mk_dataT.pivot(index = 'matchId', columns = 'teamId', values = 'minionsKilled').reset_index()
mk_data_pivT['mk_dif'] = mk_data_pivT['100'] - mk_data_pivT['200']
mk_data_pivT.columns = ['matchId', 't1mk', 't2mk', 'mkdif']
mk_data_pivT

,matchId,t1mk,t2mk,mkdif
0,8000,210,199,11
1,8001,225,196,29
2,8002,165,176,-11
3,8003,220,196,24
4,8004,249,218,31
...,...,...,...,...
1995,9995,226,208,18
1996,9996,230,203,27
1997,9997,209,211,-2
1998,9998,238,167,71


In [74]:
participants_test = pd.read_csv('../data/participants_test.csv')

In [75]:
champion_pointsT = pd.merge(participants_test, champion_mastery, on = ['summonerId', 'championId'], how = 'left' ).drop(['chestGranted', 'tokensEarned'], axis = 1)
champion_pointsT

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints
0,8000,100,1,23938,57,Fiora,114,3.0,10896.0
1,8000,100,2,8037,788,LeeSin,64,7.0,143994.0
2,8000,100,3,8444,566,Veigar,45,7.0,203012.0
3,8000,100,4,24831,542,Twitch,29,7.0,376388.0
4,8000,100,5,23890,224,Soraka,16,7.0,625009.0
...,...,...,...,...,...,...,...,...,...
19995,9999,200,6,45320,458,Mordekaiser,82,7.0,117779.0
19996,9999,200,7,45321,330,FiddleSticks,9,5.0,22922.0
19997,9999,200,8,45322,462,Tristana,18,7.0,169934.0
19998,9999,200,9,45323,256,Samira,360,4.0,19716.0


In [76]:
max_champ_ptsT = pd.DataFrame(champion_pointsT.groupby(['matchId', 'teamId'])['championPoints'].max().reset_index())
max_champ_ptsT
max_champ_ptsT = pd.merge(champion_pointsT, max_champ_ptsT, on = ['matchId', 'teamId', 'championPoints']).drop(['summonerLevel'], axis =1)
max_champ_ptsT

,matchId,teamId,participantId,summonerId,championName,championId,championLevel,championPoints
0,8000,100,5,23890,Soraka,16,7.0,625009.0
1,8000,200,10,15490,Brand,63,7.0,375328.0
2,8001,100,1,11768,Kayle,10,7.0,474445.0
3,8001,200,6,5857,Darius,122,7.0,837077.0
4,8002,100,1,39646,Poppy,78,7.0,164164.0
...,...,...,...,...,...,...,...,...
3995,9997,200,7,9838,MasterYi,11,7.0,1856187.0
3996,9998,100,1,41487,Renekton,58,7.0,728491.0
3997,9998,200,7,18389,Hecarim,120,7.0,525198.0
3998,9999,100,5,45319,Thresh,412,5.0,102891.0


In [77]:
#dataframe for max champ points by team
max_champ_pts_pivT = pd.pivot(data = max_champ_ptsT, index = 'matchId', columns = 'teamId', values = 'championPoints').reset_index()
#max_champ_pts_piv = max_champ_pts_piv.rename(columns = {'100':'t_100', '200':'t_200'}, inplace = True)
max_champ_pts_pivT.columns = ['matchId', 't1maxChPt', 't2maxChPt']
max_champ_pts_pivT['mxchmpPtDiff'] = max_champ_pts_pivT['t1maxChPt'] - max_champ_pts_pivT['t2maxChPt']
max_champ_pts_pivT

,matchId,t1maxChPt,t2maxChPt,mxchmpPtDiff
0,8000,625009.0,375328.0,249681.0
1,8001,474445.0,837077.0,-362632.0
2,8002,164164.0,74657.0,89507.0
3,8003,399780.0,35506.0,364274.0
4,8004,894152.0,179980.0,714172.0
...,...,...,...,...
1995,9995,237322.0,124155.0,113167.0
1996,9996,827163.0,717229.0,109934.0
1997,9997,226507.0,1856187.0,-1629680.0
1998,9998,728491.0,525198.0,203293.0


In [78]:
#selecting champion with highest points within each team
championT = pd.pivot(data = max_champ_ptsT, index = 'matchId', columns = 'teamId', values = 'championName').reset_index()
championT.columns = ['matchId', 't1champ', 't2champ']
championT

,matchId,t1champ,t2champ
0,8000,Soraka,Brand
1,8001,Kayle,Darius
2,8002,Poppy,Nocturne
3,8003,Elise,Malphite
4,8004,Lulu,Yorick
...,...,...,...
1995,9995,Soraka,Lissandra
1996,9996,Yasuo,Lulu
1997,9997,Zed,MasterYi
1998,9998,Renekton,Hecarim


In [79]:
featsTEST = pd.merge(gold_data_pivT, curgold_data_pivT, on = 'matchId')\
.merge(xp_data_pivT, on = 'matchId')\
.merge(mk_data_pivT, on = 'matchId')\
.merge(max_champ_pts_pivT, on = 'matchId')\
.merge(championT, on = 'matchId')

featsTEST

,matchId,t1totG,t2totG,totGdif,t1curG,t2curG,curGdif,t1xp,t2xp,xpdif,t1mk,t2mk,mkdif,t1maxChPt,t2maxChPt,mxchmpPtDiff,t1champ,t2champ
0,8000,17313,16933,380,4428,3093,1335,3619.8,3423.4,196.4,210,199,11,625009.0,375328.0,249681.0,Soraka,Brand
1,8001,17492,17533,-41,3052,3993,-941,3351.2,3305.4,45.8,225,196,29,474445.0,837077.0,-362632.0,Kayle,Darius
2,8002,16206,17480,-1274,3171,3265,-94,3328.0,3303.6,24.4,165,176,-11,164164.0,74657.0,89507.0,Poppy,Nocturne
3,8003,17784,18126,-342,5019,3271,1748,3351.8,3434.6,-82.8,220,196,24,399780.0,35506.0,364274.0,Elise,Malphite
4,8004,16341,16993,-652,4111,3013,1098,3675.4,3422.6,252.8,249,218,31,894152.0,179980.0,714172.0,Lulu,Yorick
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9995,15440,15060,380,3975,3140,835,3537.6,3569.0,-31.4,226,208,18,237322.0,124155.0,113167.0,Soraka,Lissandra
1996,9996,19123,17433,1690,4939,2288,2651,3654.6,3709.0,-54.4,230,203,27,827163.0,717229.0,109934.0,Yasuo,Lulu
1997,9997,14719,17129,-2410,4379,4154,225,3442.2,3777.6,-335.4,209,211,-2,226507.0,1856187.0,-1629680.0,Zed,MasterYi
1998,9998,18353,15134,3219,4063,3159,904,3659.6,3364.6,295.0,238,167,71,728491.0,525198.0,203293.0,Renekton,Hecarim


In [80]:
XACTTEST = featsTEST 

In [81]:
le = LabelEncoder()

for col in XACTTEST.columns.values:
    le.fit(XACTTEST[col].values)
    XACTTEST[col] = le.transform(XACTTEST[col])
    
print(XACTTEST.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   matchId       2000 non-null   int64
 1   t1totG        2000 non-null   int64
 2   t2totG        2000 non-null   int64
 3   totGdif       2000 non-null   int64
 4   t1curG        2000 non-null   int64
 5   t2curG        2000 non-null   int64
 6   curGdif       2000 non-null   int64
 7   t1xp          2000 non-null   int64
 8   t2xp          2000 non-null   int64
 9   xpdif         2000 non-null   int64
 10  t1mk          2000 non-null   int64
 11  t2mk          2000 non-null   int64
 12  mkdif         2000 non-null   int64
 13  t1maxChPt     2000 non-null   int64
 14  t2maxChPt     2000 non-null   int64
 15  mxchmpPtDiff  2000 non-null   int64
 16  t1champ       2000 non-null   int32
 17  t2champ       2000 non-null   int32
dtypes: int32(2), int64(16)
memory usage: 281.2 KB
None


In [82]:
y_predTEST = logreg.predict(XACTTEST)
y_predTEST

array([100, 200, 200, ..., 200, 100, 100], dtype=int64)

In [83]:
len(y_predTEST)

2000

In [84]:
submission = pd.read_csv('../data/sample_submission.csv')
submission

,matchId,winner
0,8000,100
1,8001,100
2,8002,100
3,8003,100
4,8004,100
...,...,...
1995,9995,100
1996,9996,100
1997,9997,100
1998,9998,100


In [85]:
submission['winner'] = y_predTEST
submission

,matchId,winner
0,8000,100
1,8001,200
2,8002,200
3,8003,200
4,8004,200
...,...,...
1995,9995,100
1996,9996,100
1997,9997,200
1998,9998,100


In [86]:
submission.to_csv('../data/submissionV6_3_26.csv', index = False)